In [1]:
import os
import sys
import skimage

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import ndimage
import time

PKG_DIR = "./"
# PKG_DIR = "/project/bioinformatics/Xiao_lab/s181141/Packages"
sys.path.append(PKG_DIR)

from DIPModels.unet import model as unet_model
from DIPModels.unet import config as unet_config
from DIPModels.unet import utils as unet_utils

from DIPModels.utils_g import utils_image
from DIPModels.utils_g import utils_keras
from DIPModels.utils_g import utils_misc

from utils_nuclei import *

%matplotlib inline 
%load_ext autoreload

#import warnings;
#warnings.simplefilter('ignore')

Using TensorFlow backend.


In [2]:
%autoreload 2
## Load training config and model
MODEL_DIR = "./UNet2D_logs/"
training_config = unet_config.Config(
    NAME='nuclei', 
    NUM_CLASSES=5,
    MODEL_INPUT_SIZE=(512, 512),
    
    ENCODER = 'ResNet',
    #FILTER_SIZE=[4, 8, 16],
    ENCODER_USE_BN = True,
    ARCHITECTURE = 'resnet101',
    
    DECODER = 'UNet',
    UPSAMPLING_MODE = 'nearest',
    DECODER_USE_BN = True,
    
    DROPOUT_RATE=0.0,
    
    # batch_size = gpu_count * images_per_gpu
    GPU_COUNT=1,
    IMAGES_PER_GPU=3,
    VALIDATION_STEPS=None, 
    STEPS_PER_EPOCH=None,
    LEARNING_RATE=1e-4, # for adam
)
training_config.display()

# Load pre-trained weights
ZF_MODEL_PATH = os.path.join(PKG_DIR, "./UNet2D_logs/nuclei_ResNet_UNet20180726T1612/nuclei_ResNet_UNet_0020.h5")
# LAST_MODEL = os.path.join(MODEL_DIR, "nuclei_unet20180522T1033", "nuclei_unet_0001.h5")

weights_path = ZF_MODEL_PATH
model = unet_model.UNet(model_dir=MODEL_DIR, config=training_config, weights_path=None)
model.keras_model.summary()


Configurations:
ARCHITECTURE                   resnet101
BATCH_SIZE                     3
DECODER                        UNet
DECODER_USE_BN                 True
DROPOUT_RATE                   0.0
ENCODER                        ResNet
ENCODER_USE_BN                 True
FILTER_SIZE                    (32, 64, 128, 256, 512, 1024)
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 3
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.0001
LOSS_WEIGHTS                   [1. 1.]
MODEL_INPUT_SIZE               (512, 512)
NAME                           nuclei
NUM_CLASSES                    5
STEPS_PER_EPOCH                None
UPSAMPLING_MODE                nearest
VALIDATION_STEPS               None
WEIGHT_DECAY                   0.0001


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to              

In [4]:
valid_dir = "./data/val/"
dataset_valid = unet_model.dataset(training_config,valid_dir)
val_sequence = unet_model.DataSequence(dataset_valid, batch_size=3, # training_config.BATCH_SIZE,
                                         config=training_config,
                                         use_border_weights=True, use_class_weights=False,
                                         border_weights_sigma=32,
                                         shuffle=True)


for x, (y1, y2) in val_sequence:
    labels, bg_scores, cl_scores = model.detect(x)
    fig, axes = plt.subplots(1, 2, figsize=[4 * 2, 4], sharex=False, sharey=False)
    ax = axes.ravel()
    ax[0].imshow(x[0])
    ax[1].imshow(labels[0])
    plt.show()

ResourceExhaustedError: OOM when allocating tensor with shape[3,3,512,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: res5c_branch2b/random_uniform/RandomUniform = RandomUniform[T=DT_INT32, dtype=DT_FLOAT, seed=87654321, seed2=1666468, _device="/job:localhost/replica:0/task:0/device:GPU:0"](decoder_3_2_conv/truncated_normal/shape)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'res5c_branch2b/random_uniform/RandomUniform', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py